In [1]:
#|default_exp dataset

In [2]:
#| export
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import torch
import pandas as pd
import numpy as np


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pdb import set_trace
class CFG:
    CACHE_PATH = Path('../data/cache')

In [4]:
fns = list((CFG.CACHE_PATH/'batch_3').glob('*.pth'))

In [ ]:
#| export
# function that loads the data from the pth file and return the data and the label as pd.DataFrame
def load_data(
    fn: Path,
    columns_event: str = ["time", "charge", "auxiliary", "x", "y", "z"],
    columns_label: str = ["azimuth", "zenith"],
):
    data = torch.load(fn)
    event = pd.DataFrame.from_records(data["event"])[columns_event]
    label = pd.DataFrame.from_records(data["target"])[columns_label]
    return event.astype(np.float32), label


class IceCubeCasheDatasetV0(Dataset):
    def __init__(self, fns, max_events = 100):
        self.fns = fns
        self.max_events = max_events

    def __len__(self):
        return len(self.fns)

    def __getitem__(self, idx):
        fn = self.fns[idx]
        event, label = load_data(fn)
        
        if self.max_events:
            event = event[:self.max_events]
        event["time"] /= event["time"].max()
        event[["x", "y", "z"]] /= 500
        event["charge"] = np.log10(event["charge"])

        event = torch.tensor(event.values)
        mask = torch.ones(len(event), dtype=torch.bool)
        label = torch.tensor(label.values, dtype=torch.float32)

        return {"event": event, "mask": mask, "label": label}


#collate_fn that pads the event and mask to the max length in the batch using pythorch pad_sequence
def collate_fn(batch):
    event = [x["event"] for x in batch]
    mask = [x["mask"] for x in batch]
    label = [x["label"] for x in batch]

    event = torch.nn.utils.rnn.pad_sequence(event, batch_first=True)
    mask = torch.nn.utils.rnn.pad_sequence(mask, batch_first=True)
    return {"event": event, "mask": mask, "label": torch.concat(label)}

In [ ]:
ds = IceCubeCasheDatasetV0(fns)
dl = DataLoader(ds, batch_size=64, shuffle=True, num_workers=4, collate_fn=collate_fn)
# for x in dl:
#     break

In [ ]:
#| export
def good_luck():
    return True

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()